# 1. Set Up Notebook and Load Data 

In [1]:
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import pandas as pd
import fnmatch
import pickle
import ipywidgets as widgets
from IPython.display import display
import sys
import os
import math
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import tensorflow as tf 
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
import math
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, losses
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Reshape, Input, Flatten, Dense, Conv2DTranspose, LeakyReLU, BatchNormalization,  LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from keras.activations import elu
from tensorflow.keras.initializers import HeUniform, GlorotUniform, HeNormal
from keras.regularizers import l2, l1


# CUSTOM Imports
def list_folders_fun(path):
    """
    DESCRIPTION: List folders in a given directory.
    INPUT: Directory path
    OUTPUT: Folder names in a given directory
    """
    folder_list = [f.name for f in Path(path).iterdir() if f.is_dir()]
    return folder_list

# Specify global path
global_path = '/Users/steven/Library/Mobile Documents/com~apple~CloudDocs/0-CfA/4-Data/Datasets'
global_folders = list_folders_fun(global_path)

# Select dataset
set_widget = widgets.Dropdown(options=global_folders[:],value=global_folders[0],description='Set :',disabled=False); set_widget

: 

: 

In [8]:
# Set ID
set_id = set_widget.value
# Select Input
files = os.listdir(f'{global_path}/{set_id}/')
input_files = [f for f in files if fnmatch.fnmatch(f, 'hist2D*24*minmax*')]
input_widget = widgets.Dropdown(options=input_files[:],value=input_files[0],description='TSNE File :',disabled=False); input_widget

Dropdown(description='TSNE File :', options=('hist2D-Full-nE16-nt24-normminmax.pkl',), value='hist2D-Full-nE16…

In [9]:
# Load features
input_file = input_widget.value
with open(f'{global_path}/{set_id}/{input_file}', 'rb') as f:
    hist_dict = pickle.load(f)
ids = list(hist_dict.keys())
histograms = hist_dict.values()
features = np.array([np.array(h) for h in histograms])
# Load properties
df_properties_input = pd.read_csv(f'{global_path}/{set_id}/properties-input-Full.csv')
df_properties_input = df_properties_input[df_properties_input['obsreg_id'].isin(list(ids))]
df_properties = df_properties_input.drop_duplicates('obsreg_id', keep='first').reset_index(drop=True)
# Data
df_label = df_properties.copy()
ID = df_label['obsreg_id'].values
X = features
x = tf.expand_dims(X, 3)
X_train, X_test, ID_train, ID_test = train_test_split(X, ID, test_size=0.1, random_state=42)
x_train = tf.expand_dims(X_train, 3)
x_test = tf.expand_dims(X_test, 3)
# Print Summary
print ('Training Set', x_train.shape)
print ('Test Set', x_test.shape)
print("Number of Features: ", len(features))
print("Number of Property Sets: ", len(df_properties))

df_label.head()

Training Set (85925, 24, 16, 1)
Test Set (9548, 24, 16, 1)
Number of Features:  95473
Number of Property Sets:  95473


,name,obsreg_id,hard_hm,hard_hs,hard_ms,var_prob_b,var_prob_h,var_prob_m,var_prob_s,var_index_b
0,2CXO J173522.0-260053,10037_123,0.354778,0.782011,0.540912,0.077612,0.092974,0.756124,NaN,0.0
1,2CXO J173403.7-260619,10037_30,0.564647,0.999375,0.999375,0.475838,0.436236,0.488032,NaN,0.0
2,2CXO J173412.7-260548,10037_40,0.033729,0.343535,0.312305,0.119011,0.438140,0.236688,0.551795,0.0
3,2CXO J173416.8-260131,10037_79,0.352280,0.801999,0.610868,0.135159,0.314770,0.834953,NaN,0.0
4,2CXO J173426.3-255911,10037_81,0.548407,0.999375,0.999375,0.299827,0.675399,0.370636,NaN,0.0


In [5]:
# Load features
input_file = input_widget.value
with open(f'{global_path}/{set_id}/{input_file}', 'rb') as f:
    hist_dict = pickle.load(f)
ids = list(hist_dict.keys())
histograms = hist_dict.values()
features = np.array([np.array(h) for h in histograms], dtype=np.float32)
features = torch.from_numpy(features)
# # features = np.array([np.array(h) for h in histograms])
# features = torch.tensor([h for h in histograms], dtype=torch.float32)

# Load properties
df_properties_input = pd.read_csv(f'{global_path}/{set_id}/properties-input-Full.csv')
df_properties_input = df_properties_input[df_properties_input['obsreg_id'].isin(list(ids))]
df_properties = df_properties_input.drop_duplicates('obsreg_id', keep='first').reset_index(drop=True)
# Data
df_label = df_properties.copy()
ID = df_label['obsreg_id'].values
X = torch.as_tensor(features, dtype=torch.float32)
x = torch.unsqueeze(torch.as_tensor(X, dtype=torch.float32), 1)
# X_train, X_test, ID_train, ID_test = train_test_split(X, ID, test_size=0.1, random_state=42)
# x_train = torch.unsqueeze(torch.tensor(X_train), 1)
# x_test = torch.unsqueeze(torch.tensor(X_test), 1)
# # Print Summary
# print ('Training Set', x_train.shape)
# print ('Test Set', x_test.shape)
# print("Number of Features: ", len(features))
# print("Number of Property Sets: ", len(df_properties))

# df_label.head()

# 3. Autoencoder Model

In [18]:
input_shape = (24, 16, 1)
in1 = 24
in2 = 16
conv_layers = 2
conv_reduce = conv_layers ** 2
encoder_layers = 7
# Define encoder
encoder = Sequential()
encoder.add(Conv2D(8, (3, 3), activation='relu', padding='same', input_shape=input_shape)) #0
encoder.add(Conv2D(8, (2, 2), strides = 2, activation='relu',padding='same'))
encoder.add(Conv2D(16, (3, 3), activation='relu', padding='same'))  #2
encoder.add(Conv2D(16, (2, 2), strides = 2, activation='relu',padding='same'))
encoder.add(Flatten())
encoder.add(Dense(96))
encoder.add(Dense(96))
encoder.add(Dense(24))
# Define decoder
decoder = Sequential()
decoder.add(Dense(96, input_shape=(24,)))
decoder.add(Dense(96))
decoder.add(Dense(96))
decoder.add(Dense(in1//conv_reduce*in2//conv_reduce*16))
decoder.add(Reshape((in1//conv_reduce, in2//conv_reduce, 16)))
decoder.add(Conv2D(16, (2, 2), activation='relu', padding='same'))
decoder.add(Conv2DTranspose(16, (2, 2), strides = 2, activation='relu',padding='same'))
decoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
decoder.add(Conv2DTranspose(8, (2, 2), strides = 2, activation='relu',padding='same'))
decoder.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))
# Combine encoder and decoder
model = Sequential([encoder, decoder])
optimizer = Adam(learning_rate = 0.001)
early_stopping = EarlyStopping(monitor='val_loss', patience=encoder_layers, verbose=1)
loss_function = 'mse'
model.compile(optimizer=optimizer, loss=loss_function)
encoder.summary()
decoder.summary()
history = model.fit(X_train, X_train, epochs=15,batch_size=64,shuffle=True,validation_split=0.15,callbacks=[early_stopping])
encoder_model = Model(inputs=model.input, outputs=model.layers[5].output)
encoder_model.set_weights(model.get_weights())

name = input('Name: ')
model.save(f'{global_path}/{set_id}/XAE-AUTOENCODER-2D-CNN-{name}.h5')
encoder_model.save(f'{global_path}/{set_id}/XAE-ENCODER-2D-CNN-{name}.h5')
with open(f'{global_path}/{set_id}/XAE-HISTORY-2D-CNN-{name}.pickle', 'wb') as file:
    pickle.dump(history.history, file)


Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_38 (Conv2D)          (None, 24, 16, 8)         80        
                                                                 
 conv2d_39 (Conv2D)          (None, 12, 8, 8)          264       
                                                                 
 conv2d_40 (Conv2D)          (None, 12, 8, 16)         1168      
                                                                 
 conv2d_41 (Conv2D)          (None, 6, 4, 16)          1040      
                                                                 
 flatten_6 (Flatten)         (None, 384)               0         
                                                                 
 dense_36 (Dense)            (None, 96)                36960     
                                                                 
 dense_37 (Dense)            (None, 96)              

2023-05-05 13:56:23.552514: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-05 13:56:23.969274: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x7fc50253d430
2023-05-05 13:56:23.969318: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x7fc50253d430
2023-05-05 13:56:23.978672: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x7fc50253d430
2023-05-05 13:56:23.978708: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x7fc50253d430
2023-05-05 13:56:23.987815: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:4

NotFoundError: Graph execution error:

Detected at node 'StatefulPartitionedCall_30' defined at (most recent call last):
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/opt/anaco

In [34]:
# Define Autoencoder Model
latent_dim = 100

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Input(in_shape_1,in_shape_2),
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dense(64, activation='relu')
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(64, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(in_shape_1*in_shape_2, activation='sigmoid'),
      layers.Reshape((in_shape_1, in_shape_2))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded
  
autoencoder = Autoencoder(latent_dim) 

# Compile Autoencoder Model
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

# Train Autoencoder Model
autoencoder.fit(X_train, X_train,
                epochs=20,
                shuffle=True,
                validation_data=(X_test, X_test))

# autoencoder.fit(X_train, X_train,
#                 epochs=50,
#                 batch_size=32,
#                 shuffle=True,
#                 validation_data=(X_train, X_train),
#                 callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Epoch 1/20


ValueError: in user code:

    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/opt/anaconda3/envs/ciao-4.15/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/var/folders/q1/jbx7qj7s0dxbh7nv02nfvc680000gn/T/__autograph_generated_fileoouzqmtq